In [1]:
from tensorboard.backend.event_processing import event_accumulator
from collections import defaultdict
import glob
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
import seaborn as sns

In [2]:
event_files = defaultdict(list)
directory = '../logs/'
log_folders = [x[0] for x in os.walk(directory)][1:]

In [3]:
for folder in log_folders:
    param_conf = tuple(folder.split('/')[-1].split('_')[1:])
    event_file = glob.glob(folder)[0]
    event_files[param_conf].append(event_file)

In [4]:
tags_ds = {'Test/Acc', 'MIA/GenGap', 'Test/Bias', 'MIA/Bacc' }

In [5]:
for k, v in event_files.items():
    metrics_ds = defaultdict(list)
    for log in v:
        ea = event_accumulator.EventAccumulator(log,size_guidance = {event_accumulator.SCALARS: 0,})
        ea.Reload()
        for tag in tags_ds:
            try:
                val = ea.Scalars(tag)
                metrics_ds[tag].append(val[-1].value)
            except:
                print('err')
            
    for k2, v2 in metrics_ds.items():
        metrics_ds[k2] = [round(np.mean(v2)*100, 2), round(np.std(v2)*100, 2)]
    
    filename = f"{directory}/processsed_logs/{k}.txt"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        f.write(str(metrics_ds) + '\n')